# simple ReAct Agent from scratch

based on https://til.simonwillison.net/llms/python-react-pattern

In [ ]:
import os

os.environ["NRP_API_KEY"] = "API key herer"

In [107]:
import re


In [137]:
from kubernetes import client, config
from kubernetes.client.exceptions import ApiException
config.load_incluster_config()

v1 = client.CoreV1Api()
apps_v1 = client.AppsV1Api()
batch_v1 = client.BatchV1Api()
networking_v1 = client.NetworkingV1Api()

Implementing a simple react pattern

In [141]:

from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key = os.environ.get("NRP_API_KEY"),
    base_url = "https://llm.nrp-nautilus.io/"
)

completion = client.chat.completions.create(
    model="gemma3",
    messages=[
        {"role": "system", "content": "You are a helpful Kubernetes assistant."},
    ],
)



In [142]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gemma3", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content

In [143]:
prompt = """
You are a Kubernetes assistant. You operate in a loop of:

Thought → Action → PAUSE → Observation

At the end of this loop, you output a final Answer.

---

**Instructions:**

- Use **Thought** to explain your reasoning based on the user's request.
- Use **Action** to call one of the tools listed below. Each Action must be followed by **PAUSE** so the system can run the tool.
- The result of the action will be passed back to you as an **Observation**.
- After processing the Observation, continue the loop.
- Stop when you have gathered enough information, and provide an **Answer**.

---

**Namespace Rule:**
- Always check if the namespace is set before performing any namespaced action.
- If it is not set, ask: *"Which namespace should I use?"*
- Then call: `set_namespace: <namespace>`

---

**When to Use `describe_*` Tools:**
- If the user mentions a specific name (e.g., "ubuntu"), check for matching resources using `list_*` tools.
- If a match is found, use the appropriate `describe_*` tool for detailed information.
- If multiple resources match the name, describe each one.
- Only use `describe_*` if you're confident about the target resource name.

---

**Available Actions:**

set_namespace:
e.g. set_namespace: kube-system
Sets the namespace for all operations.

list_pods:
list_deployments:
list_services:
list_jobs:
list_configmaps:
list_secrets:
list_pvcs:
list_replicasets:
list_statefulsets:
list_daemonsets:
list_events:
list_ingresses:
list_nodes:

Each of the above lists the corresponding resources.

describe_pod:
describe_deployment:
describe_job:
describe_service:
describe_configmap:
describe_secret:
describe_pvc:
describe_replicaset:
describe_statefulset:
describe_daemonset:
describe_ingress:
describe_node:

Each of the above describes the specified resource.

---

**Example 1:**

Question: What pods are running?

Thought: I need to check if the namespace is already set. Since it isn't, I will ask the user.
Action: set_namespace: kube-system
PAUSE

(Observation: ✅ Namespace set to 'kube-system')

Thought: Now I can list the pods in the kube-system namespace.
Action: list_pods:
PAUSE

(Observation: ['coredns-abc123', 'kube-proxy-xyz789'])

Answer: The pods currently running in kube-system are: coredns-abc123, kube-proxy-xyz789.

---

**Example 2:**

Question: What is happening with ubuntu?

Thought: The user asked about something named 'ubuntu'. I will first list pods to see if any match.
Action: list_pods:
PAUSE

(Observation: ['ubuntu-runner-xyz', 'nginx'])

Thought: A pod named 'ubuntu-runner-xyz' matches. I will describe it.
Action: describe_pod: ubuntu-runner-xyz
PAUSE

(Observation: 📋 Pod 'ubuntu-runner-xyz' phase: Running, node: node-123)

Answer: The pod 'ubuntu-runner-xyz' is currently running on node node-123.

---
"""


In [ ]:
# prompt = """
# You run in a loop of Thought, Action, PAUSE, Observation.
# At the end of the loop you output an Answer.

# Use Thought to describe your reasoning about the user's question.
# Use Action to invoke one of the available tools — then return PAUSE.
# Observation will be the result of running that action.

# Before doing anything else, always check if the namespace is already set.
# If it is not, ask the user: "Which namespace should I use?"
# Then set the namespace using the set_namespace action.
# If the user later requests a new namespace, call set_namespace again.

# Your available actions are:

# set_namespace:
# e.g. set_namespace: kube-system
# Sets the namespace for all Kubernetes operations.

# list_pods:
# e.g. list_pods:
# Lists all pods in the current namespace.

# list_deployments:
# e.g. list_deployments:
# Lists all deployments in the current namespace.

# list_services:
# e.g. list_services:
# Lists all services in the current namespace.

# list_jobs:
# e.g. list_jobs:
# Lists all jobs in the current namespace.

# list_configmaps:
# e.g. list_configmaps:
# Lists all configmaps in the current namespace.

# list_secrets:
# e.g. list_secrets:
# Lists all secrets in the current namespace.

# list_pvcs:
# e.g. list_pvcs:
# Lists all persistent volume claims in the current namespace.

# list_replicasets:
# e.g. list_replicasets:
# Lists all replicasets in the current namespace.

# list_statefulsets:
# e.g. list_statefulsets:
# Lists all statefulsets in the current namespace.

# list_daemonsets:
# e.g. list_daemonsets:
# Lists all daemonsets in the current namespace.

# list_events:
# e.g. list_events:
# Lists all recent events in the current namespace.

# list_ingresses:
# e.g. list_ingresses:
# Lists all ingress resources in the current namespace.

# list_nodes:
# e.g. list_nodes:
# Lists all nodes in the cluster (not namespaced).

# describe_pod:
# e.g. describe_pod: my-pod
# Describes detailed status of the specified pod.

# describe_deployment:
# e.g. describe_deployment: nginx-deployment
# Describes the specified deployment.

# describe_job:
# e.g. describe_job: my-batch-job
# Describes the specified job.

# describe_service:
# e.g. describe_service: backend-service
# Describes the specified service.

# describe_configmap:
# e.g. describe_configmap: my-config
# Describes the specified configmap.

# describe_secret:
# e.g. describe_secret: my-secret
# Describes metadata of the specified secret (does not reveal values).

# describe_pvc:
# e.g. describe_pvc: my-claim
# Describes the specified persistent volume claim.

# describe_replicaset:
# e.g. describe_replicaset: rs-abc123
# Describes the specified replicaset.

# describe_statefulset:
# e.g. describe_statefulset: redis-cluster
# Describes the specified statefulset.

# describe_daemonset:
# e.g. describe_daemonset: log-agent
# Describes the specified daemonset.

# describe_ingress:
# e.g. describe_ingress: web-ingress
# Describes the specified ingress.

# describe_node:
# e.g. describe_node: ip-10-0-0-42
# Describes the specified node (non-namespaced).

# Example session:

# Question: What pods are running?
# Thought: I need to check if the namespace is already set. Since it isn't, I will ask the user.
# Action: set_namespace: kube-system
# PAUSE

# You will be called again with:

# Observation: ✅ Namespace set to 'kube-system'

# Then continue:

# Thought: Now I can list the pods in the kube-system namespace.
# Action: list_pods:
# PAUSE

# Then:

# Observation: ['coredns-abc123', 'kube-proxy-xyz789']

# Answer: The pods currently running in the kube-system namespace are: coredns-abc123, kube-proxy-xyz789.
# """.strip()


In [144]:
CURRENT_NAMESPACE = None

def set_namespace(ns):
    """
    Set the global namespace for all Kubernetes operations.
    """
    global CURRENT_NAMESPACE
    CURRENT_NAMESPACE = ns.strip()
    return f"✅ Namespace set to '{CURRENT_NAMESPACE}'"

def get_namespace():
    """
    Retrieve the currently set namespace.
    Raises an error if namespace is not set.
    """
    if CURRENT_NAMESPACE is None:
        raise ValueError("❌ Namespace not set. Use `set_namespace` first.")
    return CURRENT_NAMESPACE


In [145]:
import re
from kubernetes.client.rest import ApiException

def validate_k8s_name(name):
    """
    Validate that the name follows Kubernetes RFC1123 naming convention.
    """
    pattern = r'^[a-z0-9]([-a-z0-9]*[a-z0-9])?$'
    if not re.match(pattern, name):
        raise ValueError(f"❌ Invalid Kubernetes resource name: '{name}'. Must match RFC1123 format.")
    return name


# ---------- LIST FUNCTIONS ----------

def list_pods(_=None):
    namespace = get_namespace()
    pods = v1.list_namespaced_pod(namespace=namespace)
    return [pod.metadata.name for pod in pods.items]

def list_deployments(_=None):
    namespace = get_namespace()
    deployments = apps_v1.list_namespaced_deployment(namespace=namespace)
    return [d.metadata.name for d in deployments.items]

def list_services(_=None):
    namespace = get_namespace()
    services = v1.list_namespaced_service(namespace=namespace)
    return [s.metadata.name for s in services.items]

def list_jobs(_=None):
    namespace = get_namespace()
    jobs = batch_v1.list_namespaced_job(namespace=namespace)
    return [j.metadata.name for j in jobs.items]

def list_configmaps(_=None):
    namespace = get_namespace()
    cms = v1.list_namespaced_config_map(namespace=namespace)
    return [cm.metadata.name for cm in cms.items]

def list_secrets(_=None):
    namespace = get_namespace()
    secrets = v1.list_namespaced_secret(namespace=namespace)
    return [s.metadata.name for s in secrets.items]

def list_pvcs(_=None):
    namespace = get_namespace()
    pvcs = v1.list_namespaced_persistent_volume_claim(namespace=namespace)
    return [p.metadata.name for p in pvcs.items]

def list_replicasets(_=None):
    namespace = get_namespace()
    rsets = apps_v1.list_namespaced_replica_set(namespace=namespace)
    return [r.metadata.name for r in rsets.items]

def list_statefulsets(_=None):
    namespace = get_namespace()
    ssets = apps_v1.list_namespaced_stateful_set(namespace=namespace)
    return [s.metadata.name for s in ssets.items]

def list_daemonsets(_=None):
    namespace = get_namespace()
    dsets = apps_v1.list_namespaced_daemon_set(namespace=namespace)
    return [d.metadata.name for d in dsets.items]

def list_ingresses(_=None):
    namespace = get_namespace()
    ingresses = networking_v1.list_namespaced_ingress(namespace=namespace)
    return [i.metadata.name for i in ingresses.items]

def list_events(_=None):
    namespace = get_namespace()
    events = v1.list_namespaced_event(namespace=namespace)
    return [f"{e.last_timestamp}: {e.message}" for e in events.items]

def list_nodes(_=None):
    nodes = v1.list_node()
    return [n.metadata.name for n in nodes.items]


# ---------- DESCRIBE FUNCTIONS ----------

def describe_pod(name):
    name = name.strip()
    namespace = get_namespace()
    pod = v1.read_namespaced_pod(name=name, namespace=namespace)
    return f"📋 Pod '{name}' phase: {pod.status.phase}, node: {pod.spec.node_name}"

def describe_deployment(name):
    name = name.strip()
    namespace = get_namespace()
    dep = apps_v1.read_namespaced_deployment(name=name, namespace=namespace)
    return f"📦 Deployment '{name}' has {dep.status.replicas or 0} replicas and {dep.status.ready_replicas or 0} ready."

def describe_service(name):
    name = name.strip()
    namespace = get_namespace()
    svc = v1.read_namespaced_service(name=name, namespace=namespace)
    return f"🌐 Service '{name}' type: {svc.spec.type}, cluster IP: {svc.spec.cluster_ip}"

def describe_job(name):
    name = name.strip()
    namespace = get_namespace()
    job = batch_v1.read_namespaced_job(name=name, namespace=namespace)
    return f"⚙️ Job '{name}' completions: {job.status.succeeded or 0}, active: {job.status.active or 0}"

def describe_configmap(name):
    name = name.strip()
    namespace = get_namespace()
    cm = v1.read_namespaced_config_map(name=name, namespace=namespace)
    keys = list(cm.data.keys()) if cm.data else []
    return f"🗂️ ConfigMap '{name}' has keys: {keys}"

def describe_secret(name):
    name = name.strip()
    namespace = get_namespace()
    sec = v1.read_namespaced_secret(name=name, namespace=namespace)
    keys = list(sec.data.keys()) if sec.data else []
    return f"🔒 Secret '{name}' contains {len(keys)} keys (values hidden)"

def describe_pvc(name):
    name = name.strip()
    namespace = get_namespace()
    pvc = v1.read_namespaced_persistent_volume_claim(name=name, namespace=namespace)
    return f"💾 PVC '{name}' status: {pvc.status.phase}, capacity: {pvc.status.capacity.get('storage')}"

def describe_replicaset(name):
    name = name.strip()
    namespace = get_namespace()
    rs = apps_v1.read_namespaced_replica_set(name=name, namespace=namespace)
    return f"📎 ReplicaSet '{name}' replicas: {rs.status.replicas}, ready: {rs.status.ready_replicas}"

def describe_statefulset(name):
    name = name.strip()
    namespace = get_namespace()
    ss = apps_v1.read_namespaced_stateful_set(name=name, namespace=namespace)
    return f"📘 StatefulSet '{name}' replicas: {ss.status.replicas}, ready: {ss.status.ready_replicas}"

def describe_daemonset(name):
    name = name.strip()
    namespace = get_namespace()
    ds = apps_v1.read_namespaced_daemon_set(name=name, namespace=namespace)
    return f"🔁 DaemonSet '{name}' scheduled: {ds.status.current_number_scheduled}, ready: {ds.status.number_ready}"



def describe_ingress(name):
    name = name.strip()
    namespace = get_namespace()
    try:
        ing = networking_v1.read_namespaced_ingress(name=name, namespace=namespace)
        hosts = [rule.host for rule in ing.spec.rules] if ing.spec.rules else []
        services = []
        for rule in ing.spec.rules or []:
            if rule.http:
                for path in rule.http.paths:
                    if path.backend and path.backend.service:
                        services.append(path.backend.service.name)
        return f"🚪 Ingress '{name}' exposes hosts: {hosts or '[]'} and forwards to services: {services or '[]'}"
    except ApiException as e:
        if e.status == 404:
            return f"❌ Ingress '{name}' not found in namespace '{namespace}'."
        raise


def describe_node(name):
    name = name.strip()
    node = v1.read_node(name=name)
    return f"🖥️ Node '{name}' labels: {node.metadata.labels}"


In [146]:
known_actions = {
    # Namespace control
    "set_namespace": set_namespace,

    # LIST actions
    "list_pods": list_pods,
    "list_deployments": list_deployments,
    "list_services": list_services,
    "list_jobs": list_jobs,
    "list_configmaps": list_configmaps,
    "list_secrets": list_secrets,
    "list_pvcs": list_pvcs,
    "list_replicasets": list_replicasets,
    "list_statefulsets": list_statefulsets,
    "list_daemonsets": list_daemonsets,
    "list_ingresses": list_ingresses,
    "list_events": list_events,
    "list_nodes": list_nodes,

    # DESCRIBE actions
    "describe_pod": describe_pod,
    "describe_deployment": describe_deployment,
    "describe_service": describe_service,
    "describe_job": describe_job,
    "describe_configmap": describe_configmap,
    "describe_secret": describe_secret,
    "describe_pvc": describe_pvc,
    "describe_replicaset": describe_replicaset,
    "describe_statefulset": describe_statefulset,
    "describe_daemonset": describe_daemonset,
    "describe_ingress": describe_ingress,
    "describe_node": describe_node,
}



In [115]:
abot =Agent(prompt)

In [116]:
result = abot("in gsoc namespace get me the list of pods")
print(result)

Thought: I need to check if the namespace is already set. Since it isn't, I will ask the user.
Action: set_namespace: gsoc
PAUSE



In [117]:
result = list_pods()
print(result)

ValueError: ❌ Namespace not set. Use `set_namespace` first.

In [ ]:
result

['agno-deployment-55c55964db-hhwh5',
 'my-postgres-cluster-0',
 's1-849c54645b-xcqdm',
 'shellshock-cluster-0']

In [ ]:
next_prompt = "Observation: {}".format(result)

In [ ]:
abot = Agent(prompt)

In [ ]:
abot(next_prompt)

"Thought: The `list_pods` action returned a list of pods in the 'gsoc' namespace. I need to present this information to the user in a clear and understandable format.\nAnswer: The pods currently running in the gsoc namespace are: agno-deployment-55c55964db-hhwh5, my-postgres-cluster-0, s1-849c54645b-xcqdm, shellshock-cluster-0."

## Add Loop

In [147]:
action_re = re.compile(r'^Action: (\w+):(.*)$')

In [148]:
def query(question, max_turns=15):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        print(f"\n--- Turn {i+1} ---")
        i += 1
        print("Prompt to bot:", next_prompt)

        result = bot(next_prompt)
        print("Bot response:\n", result)

        actions = [
            action_re.match(a)
            for a in result.split('\n')
            if action_re.match(a)
        ]

        if actions:
            for action_match in actions:
                action, action_input = action_match.groups()
                if action not in known_actions:
                    raise Exception(f"Unknown action: {action}: {action_input}")
                print(f" -- Running action '{action}' with input '{action_input}'")
                observation = known_actions[action](action_input)
                print("Observation:", observation)
                next_prompt = f"Observation: {observation}"
        else:
            print("No more actions. Halting.")
            return


In [149]:
question = """I want to everything related to the name ubuntu in gsoc namespace"""

query(question)


--- Turn 1 ---
Prompt to bot: I want to everything related to the name ubuntu in gsoc namespace
Bot response:
 Thought: The user is asking for information about resources named 'ubuntu' within the 'gsoc' namespace. First, I need to set the namespace to 'gsoc'.
Action: set_namespace: gsoc
PAUSE

 -- Running action 'set_namespace' with input ' gsoc'
Observation: ✅ Namespace set to 'gsoc'

--- Turn 2 ---
Prompt to bot: Observation: ✅ Namespace set to 'gsoc'
Bot response:
 Thought: Now that the namespace is set to 'gsoc', I will start by listing pods to see if any resources named 'ubuntu' exist.
Action: list_pods:
PAUSE
 -- Running action 'list_pods' with input ''
Observation: ['agno-deployment-55c55964db-hhwh5', 'my-postgres-cluster-0', 's1-849c54645b-xcqdm', 'shellshock-cluster-0']

--- Turn 3 ---
Prompt to bot: Observation: ['agno-deployment-55c55964db-hhwh5', 'my-postgres-cluster-0', 's1-849c54645b-xcqdm', 'shellshock-cluster-0']
Bot response:
 Thought: The list of pods doesn't contai

In [150]:
question = """I want to know which services are connected which ingresses in the gsoc namespace"""

query(question)


--- Turn 1 ---
Prompt to bot: I want to know which services are connected which ingresses in the gsoc namespace
Bot response:
 Thought: The user is asking about the relationship between services and ingresses in the 'gsoc' namespace. First, I need to check if the namespace is set. It is not, so I will ask the user to set it.
Action: set_namespace: gsoc
PAUSE

 -- Running action 'set_namespace' with input ' gsoc'
Observation: ✅ Namespace set to 'gsoc'

--- Turn 2 ---
Prompt to bot: Observation: ✅ Namespace set to 'gsoc'
Bot response:
 Thought: Now that the namespace is set to 'gsoc', I need to find the ingresses and services within that namespace. I will start by listing the ingresses to see what exists.
Action: list_ingresses:
PAUSE
 -- Running action 'list_ingresses' with input ''
Observation: ['egl-03ac-1751044146', 'egl-03ac-1751044880', 'egl-03ac-1751045488', 'egl-03ac-1751048534', 'egl-03ac-1751052239', 'egl-03ac-1751053000', 'egl-03ac-1751297258', 'egl-03ac-1751298921', 'egl-03a

In [151]:
question = """can u describe all the pods in the gsoc namespace"""

query(question)


--- Turn 1 ---
Prompt to bot: can u describe all the pods in the gsoc namespace
Bot response:
 Thought: The user wants me to describe all pods in the 'gsoc' namespace. First, I need to check if the namespace is set. It is not, so I need to ask the user which namespace to use.
Action: set_namespace: gsoc
PAUSE

 -- Running action 'set_namespace' with input ' gsoc'
Observation: ✅ Namespace set to 'gsoc'

--- Turn 2 ---
Prompt to bot: Observation: ✅ Namespace set to 'gsoc'
Bot response:
 Thought: Now that the namespace is set to 'gsoc', I need to list the pods in that namespace to get their names. Then, I will describe each pod individually.
Action: list_pods:
PAUSE
 -- Running action 'list_pods' with input ''
Observation: ['agno-deployment-55c55964db-hhwh5', 'my-postgres-cluster-0', 's1-849c54645b-xcqdm', 'shellshock-cluster-0']

--- Turn 3 ---
Prompt to bot: Observation: ['agno-deployment-55c55964db-hhwh5', 'my-postgres-cluster-0', 's1-849c54645b-xcqdm', 'shellshock-cluster-0']
Bot res